In [1]:
import sys
sys.path.append("../src")

from models.s4d import S4DModel, S4DConfig, S4DModelForHourlySeries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

df = pd.read_csv("../data/merged_data.csv", parse_dates=["datetime"])
df = df.sort_values("datetime")
df = df.dropna()

features = [
    "temperature_2m", "wind_speed_180m", "wind_speed_120m", "direct_radiation",
    "quantity_biomass", "quantity_fossil_gas", "quantity_fossil_hard_coal",
    "quantity_hydro_run_of_river", "quantity_nuclear", "quantity_solar",
    "quantity_waste", "quantity_wind_offshore", "quantity_wind_onshore",
    "quantity_other", "quantity_MW"
]
target = "price_EUR_MWh"

scaler_x = StandardScaler()
scaler_y = StandardScaler()

X = scaler_x.fit_transform(df[features])
y = scaler_y.fit_transform(df[[target]])


In [ ]:
class TimeSeriesDataset(Dataset):
    """
    Custom Dataset for time series data with fixed sequence length.
    """
    def __init__(self, X, y, seq_len):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.seq_len = seq_len

    def __len__(self):
        return len(self.X) - self.seq_len

    def __getitem__(self, idx):
        x = self.X[idx : idx + sequence_length]          # input sequence of length 24
        t = self.y[idx + sequence_length : idx + 2*sequence_length]  # target: next 24 steps
        return x, t

In [3]:
# Splitting the dataset into training and validation sets
# Assuming the data is hourly and we want to validate on the last 30 days
sequence_length = 24 # 24 hours = 1 day
val_days = 30
val_size = val_days * 24

train_X = X[:-val_size]
train_y = y[:-val_size]
val_X = X[-val_size - sequence_length:]
val_y = y[-val_size - sequence_length:]

train_dataset = TimeSeriesDataset(train_X, train_y, sequence_length)
val_dataset = TimeSeriesDataset(val_X, val_y, sequence_length)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [4]:
# Model Config

config = S4DConfig(    
    state_size   = 64,
    hidden_size  = 32,
    num_layers   = 1,
    dropout      = 0.1,
)

# Model, optimizer, and loss function
model = S4DModelForHourlySeries(config) #.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss() #.to("cuda")

# Training the model
n_epochs = 30
model.train()
for epoch in range(n_epochs):
    epoch_loss = 0
    for x, t in train_loader:
        # x, t = x.cuda(), t.cuda()
        optimizer.zero_grad()
        preds = model(x, mode="convolutional")
        print(f"preds.shape = {preds.shape}")  # should be (batch, 24)
        print(f"t.shape = {t.shape}")          # should be (batch, 24) or (batch, 24, 1)

        # If t has an extra last dim, squeeze it
        t_squeezed = t.squeeze(-1)
        print(f"t_squeezed.shape = {t_squeezed.shape}")
        loss = criterion(preds, t.squeeze(-1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x, t in val_loader:
            # x, t = x.cuda(), t.cuda()
            preds = model(x)
            loss = criterion(preds, t.squeeze(-1))
            val_loss += loss.item()
    model.train()

    print(f"Epoch {epoch+1}, Train Loss: {epoch_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")

preds.shape = torch.Size([32, 24])
t.shape = torch.Size([32, 1])
t_squeezed.shape = torch.Size([32])


d:\Uni\Zurich\Semester 1\Advanced Machine Learning\project\aml2025-group-15-main\aml2025-group-15\.venv\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 24])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (24) must match the size of tensor b (32) at non-singleton dimension 1